In [1]:
from tensorflow import keras
import tensorflow_datasets as tfds
import tensorflow as tf
from keras.layers import (Input, Dense, Conv2D, Conv2DTranspose, BatchNormalization, UpSampling2D, LeakyReLU,
                          Dropout, Flatten, Reshape, Lambda, MaxPool2D, Concatenate, add, Cropping2D)
from keras.models import Model
from keras.optimizers import RMSprop, Adam
from math import log2
import matplotlib.pyplot as plt
from random import randint
import numpy as np

In [2]:
# Константы
img_side = 70

# Разбиваем датасет на тренировочную группу и группу валидации
def init_data_with_batch_size(batch_size):
    global train_data, val_data
    train_data = tf.keras.utils.image_dataset_from_directory(
        # Чтобы использовать "new_flowers" (расширенный датасет) надо запустить increasing_data.py
        "new_flowers",
        image_size=(img_side, img_side),
        shuffle=True,
        batch_size=batch_size,

        subset="training",
        validation_split=0.1,
        seed=123,
    )
    val_data = tf.keras.utils.image_dataset_from_directory(
        # Чтобы использовать "new_flowers" (расширенный датасет) надо запустить increasing_data.py
        "new_flowers",
        image_size=(img_side, img_side),
        shuffle=True,
        batch_size=batch_size,

        subset="validation",
        validation_split=0.1,
        seed=123,
    )

    # Убираем лейблы (т.к. у нас задача не распознавать изображения)
    train_data = train_data.map(lambda x, y: (x/255., x/255.))
    val_data = val_data.map(lambda x, y: (x/255., x/255.))

    return train_data, val_data

Модель

In [3]:
# Константы
filters = 32
hidden_units = 4**3  # Только степень 4 (4**?) (Размер "смыслового вектора")
amount_encode_layers = 3
amount_decode_layers = 3
max_boudle_residual_layers = 3  # Сколько слоёв с остаточноым обучением перед уменьшением/увеличением картинки
"""
filters                     Слабое  влияние на количество параметров
hidden_units                Никакое влияние на количество параметров
amount_encode_layers        Сильное влияние на количество параметров
amount_decode_layers        Сильное влияние на количество параметров
max_boudle_residual_layers  Слабое  влияние на количество параметров
"""

core_size = (3, 3)
input_img = Input(shape=(img_side, img_side, 3))

"""Энкодер"""
# Это надо чтобы первый слой мог сложиться со следующим (при помощи add([x_temp, x]) )
# (momentum - параметр расчета скользящего среднего и дисперсии)
x = BatchNormalization(momentum=0.8)(input_img)
x_temp = Conv2D(filters, core_size, activation=LeakyReLU(0.1), padding="same")(x)

for i in range(amount_encode_layers):
    for _ in range(max_boudle_residual_layers):
        x = Conv2D(filters // 2**i, core_size, activation=LeakyReLU(0.1), padding="same")(x_temp)
        x = Conv2D(filters // 2**i, core_size, activation=LeakyReLU(0.1), padding="same")(x)
        x_temp = add([x_temp, x])

    x_temp = MaxPool2D((2, 2))(x_temp)
    x_temp = Conv2D(filters // 2**(i + 1), core_size, activation=LeakyReLU(0.1), padding="same")(x_temp)
    x_temp = BatchNormalization(momentum=0.8)(x_temp)

# Превращаем сжатую картинку в "смысловой вектор"
x = Flatten()(x_temp)
x = Dense(hidden_units, activation=LeakyReLU(0.1))(x)
x = Reshape((int(hidden_units**.5), int(hidden_units**.5), 1))(x)
x_temp = BatchNormalization(momentum=0.8)(x)


"""Декодер"""
# Расширяем карту признаков, увеличиваем картинку и количество фильтров
for i in range(amount_decode_layers, 0, -1):
    for _ in range(max_boudle_residual_layers):
        x = Conv2D(filters // 2**i, core_size, activation=LeakyReLU(0.1), padding="same")(x)
        x = Conv2D(filters // 2**i, core_size, activation=LeakyReLU(0.1), padding="same")(x)
        # x_temp = add([x_temp, x])

    # Чтобы количество фильтров совпадало (для остаточного обучения)
    # x_temp = Conv2DTranspose(filters // 2**(i - 1),
    #                          core_size,
    #                          activation=LeakyReLU(0.1),
    #                          padding="same",
    #                          strides=2
    #                          )(x_temp)
    x = UpSampling2D()(x)
    x_temp = BatchNormalization(momentum=0.8)(x)

# Сжимаем количество фильтров
for i in range(int(log2(filters))):
    x = Conv2D(filters // 2**(i+1), core_size, activation=LeakyReLU(0.1))(x_temp)
    x = Conv2D(filters // 2**(i+1), core_size, activation=LeakyReLU(0.1))(x)
    x_temp = BatchNormalization(momentum=0.8)(x)

# Строим финальное изображение
x = Flatten()(x_temp)
x = Dense(img_side**2*3, activation="sigmoid")(x)
x = Reshape((img_side, img_side, 3))(x)

# # Постобработка (сглаживание шума)
# x = Conv2D(3, (13, 13), activation="sigmoid", padding="same")(x)

# Модель
output_img = x
vae = Model(input_img, output_img, name="vae")
vae.compile(
    optimizer=keras.optimizers.Adam(3e-3),
    loss="binary_crossentropy",
    loss_weights=[1000],
)

vae.summary()

Model: "vae"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 70, 70, 3)]  0           []                               
                                                                                                  
 batch_normalization (BatchNorm  (None, 70, 70, 3)   12          ['input_1[0][0]']                
 alization)                                                                                       
                                                                                                  
 conv2d (Conv2D)                (None, 70, 70, 32)   896         ['batch_normalization[0][0]']    
                                                                                                  
 conv2d_1 (Conv2D)              (None, 70, 70, 32)   9248        ['conv2d[0][0]']               

In [ ]:
train_data, _ = init_data_with_batch_size(128)
vae.fit(
    train_data,
    epochs=100,
)

Found 37799 files belonging to 5 classes.
Using 34020 files for training.
Found 37799 files belonging to 5 classes.
Using 3779 files for validation.
Epoch 1/100
 88/266 [========>.....................] - ETA: 1:12 - loss: 613.5624

In [ ]:
def show_row_images(raw_data, title):
    data = np.array([i[0][0] for count, i in enumerate(raw_data)
                         if count < 32])
    generated_images = vae.predict(data, verbose=False)

    num_images = 4

    plt.figure(figsize=(20, 11))

    for _ in range(num_images):
        random_num = randint(0, 32-1)

        # Оригинальное изображение
        plt.subplot(2, num_images, _ + 1)
        plt.imshow(data[random_num])
        plt.gray()
        plt.title(title)
        plt.axis("off")

        # Сгенерированное изображение
        plt.subplot(2, num_images, _ + num_images + 1)
        plt.imshow(generated_images[random_num])
        plt.gray()
        plt.title("Generated")
        plt.axis("off")
    plt.tight_layout()
    plt.show()

_, val_data = init_data_with_batch_size(1)
for _ in range(3):
    show_row_images(val_data, "Validation")

# print(f"Dropout: {dropout}")

train_data, _ = init_data_with_batch_size(1)
for _ in range(3):
    show_row_images(train_data, "Train Data")

In [ ]:
"""Выводим Архитектуру"""
img_file = "architecture.png"
tf.keras.utils.plot_model(vae, to_file=img_file, show_shapes=True, show_layer_names=False)